In [ ]:
import re

import numpy as np

import pandas as pd

from pprint import pprint

import gensim

import gensim.corpora as corpora

from gensim.utils import simple_preprocess

from nltk.corpus import stopwords

from gensim.models import CoherenceModel

import spacy

# import pyLDAvis

# import pyLDAvis.gensim_models

import matplotlib.pyplot as plt

import nltk

import spacy

nltk.download('stopwords')

nlp=spacy.load('en_core_web_sm',disable=['parser', 'ner'])



#importing the Stopwords to use them

stop_words = stopwords.words('english')

stop_words.extend(['from', 'subject', 're', 'edu', 'use','for'])


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv('dataset.csv')
print(data)

       Unnamed: 0                       Date             User  \
0               0  2023-03-09 13:02:01+00:00     devonclimate   
1               1  2023-03-09 13:01:22+00:00  colpacpackaging   
2               2  2023-03-09 13:00:05+00:00     OxonRecycles   
3               3  2023-03-09 13:00:02+00:00    recycleforgsy   
4               4  2023-03-09 13:00:00+00:00   MichaelsEnergy   
...           ...                        ...              ...   
49995       49995  2022-11-07 21:37:46+00:00      CpmTechlive   
49996       49996  2022-11-07 21:31:14+00:00   PHXPublicWorks   
49997       49997  2022-11-07 21:21:33+00:00        LumpyPete   
49998       49998  2022-11-07 21:15:05+00:00       CalRecycle   
49999       49999  2022-11-07 21:13:28+00:00          JonW323   

                                                   Tweet  
0      Producing uneaten food squanders a whole host ...  
1      Our Modified Atmosphere Packaging (M.A.P.) tec...  
2      Could you take action on food waste

In [ ]:
data =data.iloc[:,3]
print(data)

0        Producing uneaten food squanders a whole host ...
1        Our Modified Atmosphere Packaging (M.A.P.) tec...
2        Could you take action on food waste?\n♻ We're ...
3        Did you know? If food waste were a country, it...
4        🌮🗑️🍓 So much waste! This week we're ranting ab...
                               ...                        
49995    Get the most out of your groceries: How to pre...
49996    We were honored to have @MayorGallego join us ...
49997                         @AvonandsomerRob Food waste.
49998    Make recycling food scraps fun! Join Californi...
49999    @999AndyBell @Advice_GrainUK Supermarkets are ...
Name: Tweet, Length: 50000, dtype: object


In [ ]:
dt = []
for j in data:
  dt.append(str(j))
# print(list1)

In [ ]:
dt = [re.sub('\S*@\S*\s?', '', sent) for sent in dt]

In [ ]:
dt = [re.sub('\s+', ' ', sent) for sent in dt]

dt = [re.sub("\'", "", sent) for sent in dt]

In [ ]:
def tokeniz(sentences):

    for sentence in sentences:

         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

processed_data = list(tokeniz(dt))

In [ ]:
bigram = gensim.models.Phrases(processed_data, min_count=5, threshold=100)

trigram = gensim.models.Phrases(bigram[processed_data], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.9/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def remove_stopwords(texts):

    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
def create_bigrams(texts):

    return [bigram_mod[doc] for doc in texts]

#function to create trigrams

def create_trigrams(texts):

    [trigram_mod[bigram_mod[doc]] for doc in texts]



#function for lemmatization

def lemmatize(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):

    texts_op = []

    for sent in texts:

        doc = nlp(" ".join(sent))

        texts_op.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])

    return texts_op

In [ ]:
data_wo_stopwords = remove_stopwords(processed_data)

data_bigrams = create_bigrams(data_wo_stopwords)

data_lemmatized = lemmatize(data_bigrams, allowed_postags=[ 'NOUN', 'ADJ', 'VERB'])

In [ ]:
print(data_lemmatized[:3])

[['produce', 'uneaten', 'food', 'squander', 'whole', 'host', 'resource', 'generate', 'greenhousegase', 'stage', 'begin', 'reduce', 'food', 'waste', 'think', 'positive', 'change', 'make', 'see', 'top_tip', 'website', 'https'], ['modify', 'atmosphere', 'packaging', 'technique', 'replace', 'oxygen', 'protective', 'gas', 'mix', 'sealed', 'pack', 'life', 'food', 'day', 'amp', 'reduce', 'risk'], ['take', 'action', 'food', 'waste', 'recruit', 'food', 'waste', 'recycling', 'engagement', 'officer', 's', 'week', 'leave', 'apply', 'click', 'detail', 'jsmxvylh']]


In [ ]:

gensim_dictionary = corpora.Dictionary(data_lemmatized)



texts = data_lemmatized



#building a corpus for the topic model

gensim_corpus = [gensim_dictionary.doc2bow(text) for text in texts]

In [ ]:
[[(gensim_dictionary[id], freq) for id, freq in cp] for cp in gensim_corpus[:4]]

[[('begin', 1),
  ('change', 1),
  ('food', 2),
  ('generate', 1),
  ('greenhousegase', 1),
  ('host', 1),
  ('https', 1),
  ('make', 1),
  ('positive', 1),
  ('produce', 1),
  ('reduce', 1),
  ('resource', 1),
  ('see', 1),
  ('squander', 1),
  ('stage', 1),
  ('think', 1),
  ('top_tip', 1),
  ('uneaten', 1),
  ('waste', 1),
  ('website', 1),
  ('whole', 1)],
 [('food', 1),
  ('reduce', 1),
  ('amp', 1),
  ('atmosphere', 1),
  ('day', 1),
  ('gas', 1),
  ('life', 1),
  ('mix', 1),
  ('modify', 1),
  ('oxygen', 1),
  ('pack', 1),
  ('packaging', 1),
  ('protective', 1),
  ('replace', 1),
  ('risk', 1),
  ('sealed', 1),
  ('technique', 1)],
 [('food', 2),
  ('waste', 2),
  ('action', 1),
  ('apply', 1),
  ('click', 1),
  ('detail', 1),
  ('engagement', 1),
  ('jsmxvylh', 1),
  ('leave', 1),
  ('officer', 1),
  ('recruit', 1),
  ('recycling', 1),
  ('s', 1),
  ('take', 1),
  ('week', 1)],
 [('food', 2),
  ('https', 1),
  ('reduce', 2),
  ('waste', 2),
  ('big', 1),
  ('carbon_footprint',

In [ ]:
#LSI MODEL
lsi_model = gensim.models.lsimodel.LsiModel(
   corpus=gensim_corpus, id2word=gensim_dictionary, num_topics=20,chunksize=100
)

In [ ]:
pprint(lsi_model.print_topics())

[(0,
  '0.751*"food" + 0.587*"waste" + 0.151*"https" + 0.120*"reduce" + 0.084*"co" '
  '+ 0.058*"help" + 0.055*"amp" + 0.051*"make" + 0.041*"go" + 0.034*"year"'),
 (1,
  '0.832*"https" + 0.432*"co" + -0.190*"food" + 0.132*"reduce" + '
  '0.117*"foodwaste" + 0.108*"amp" + -0.102*"waste" + 0.081*"help" + '
  '0.036*"learn" + 0.035*"sustainability"'),
 (2,
  '0.779*"waste" + -0.595*"food" + -0.058*"foodwaste" + 0.050*"recycling" + '
  '-0.049*"help" + 0.048*"compost" + 0.044*"collection" + 0.038*"recycle" + '
  '-0.035*"reduce" + 0.030*"plastic"'),
 (3,
  '-0.835*"reduce" + -0.288*"amp" + -0.261*"help" + 0.211*"https" + '
  '0.138*"food" + -0.132*"foodwaste" + 0.110*"co" + -0.075*"way" + '
  '-0.067*"make" + -0.051*"tip"'),
 (4,
  '0.899*"amp" + -0.347*"reduce" + 0.089*"foodwaste" + -0.073*"co" + '
  '0.067*"go" + 0.062*"compost" + 0.058*"make" + 0.052*"people" + 0.048*"get" '
  '+ -0.047*"help"'),
 (5,
  '0.647*"make" + -0.302*"co" + 0.286*"go" + 0.230*"help" + 0.217*"get" + '
  '-0.216*

In [ ]:
import time
time.clock = time.time

In [ ]:
# HDP MODEL
hdp_model = gensim.models.hdpmodel.HdpModel(corpus=gensim_corpus, id2word=gensim_dictionary)



#viewing topics

pprint(hdp_model.print_topics())

[(0,
  '0.078*food + 0.062*waste + 0.018*https + 0.014*reduce + 0.010*co + '
  '0.008*amp + 0.008*help + 0.007*make + 0.005*go + 0.005*foodwaste'),
 (1,
  '0.063*food + 0.052*waste + 0.017*https + 0.011*reduce + 0.009*co + '
  '0.007*amp + 0.006*help + 0.005*make + 0.005*foodwaste + 0.004*compost'),
 (2,
  '0.042*food + 0.033*waste + 0.010*https + 0.007*reduce + 0.006*co + '
  '0.005*amp + 0.004*make + 0.004*help + 0.003*go + 0.003*foodwaste'),
 (3,
  '0.042*food + 0.033*waste + 0.010*https + 0.007*reduce + 0.005*co + '
  '0.004*amp + 0.004*make + 0.003*help + 0.003*go + 0.003*foodwaste'),
 (4,
  '0.036*food + 0.029*waste + 0.010*https + 0.006*reduce + 0.005*co + '
  '0.005*amp + 0.003*help + 0.003*make + 0.003*foodwaste + 0.002*go'),
 (5,
  '0.037*food + 0.030*waste + 0.009*https + 0.006*reduce + 0.005*co + '
  '0.004*amp + 0.003*help + 0.003*make + 0.002*go + 0.002*foodwaste'),
 (6,
  '0.036*food + 0.028*waste + 0.010*https + 0.006*reduce + 0.005*co + '
  '0.004*amp + 0.003*make + 0.

In [ ]:
list1 = []
for j in lsi_model.print_topics():
  list2 = []
  c=3
  for x in range(len(j[1])):
    if(j[1][x]=='*' and c>0):
      c= c-1
      s = ""
      while(j[1][x]!=' ' and x<len(j[1])-1):
        x= x+1
        if(j[1][x]==' '):
          break
        s+=j[1][x]

      list2.append(s)
  list1.append(list2)
print(list1)


[['"food"', '"waste"', '"https"'], ['"https"', '"co"', '"food"'], ['"waste"', '"food"', '"foodwaste"'], ['"reduce"', '"amp"', '"help"'], ['"amp"', '"reduce"', '"foodwaste"'], ['"make"', '"co"', '"go"'], ['"co"', '"https"', '"make"'], ['"help"', '"make"', '"reduce"'], ['"go"', '"make"', '"compost"'], ['"compost"', '"go"', '"eat"'], ['"foodwaste"', '"year"', '"go"'], ['"get"', '"go"', '"week"'], ['"eat"', '"recycling"', '"bag"'], ['"people"', '"year"', '"save"'], ['"get"', '"people"', '"year"'], ['"year"', '"get"', '"good"'], ['"take"', '"save"', '"use"'], ['"take"', '"use"', '"save"'], ['"week"', '"time"', '"recycling"'], ['"use"', '"time"', '"week"']]


In [ ]:

from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
def compute_coherence_UMass(corpus, dictionary, k):
   coherence = CoherenceModel(model=lsi_model,
                              corpus=corpus,
                              dictionary=dictionary,
                              coherence='u_mass')
   return coherence.get_coherence()
coherenceList_UMass = []
numTopicsList = [20,100,200,300,400,500,800,1000, 1500]
for k in numTopicsList:
   c_UMass = compute_coherence_UMass(gensim_corpus, gensim_dictionary, 10)
   coherenceList_UMass.append(c_UMass)

In [ ]:
print(coherenceList_UMass)


[-3.750302762174767, -3.750302762174767, -3.750302762174767, -3.750302762174767, -3.750302762174767, -3.750302762174767, -3.750302762174767, -3.750302762174767, -3.750302762174767]


In [ ]:
dh = pd.DataFrame(list1)
dh = dh.T
print(dh)

        0        1            2         3            4       5        6   \
0   "food"  "https"      "waste"  "reduce"        "amp"  "make"     "co"   
1  "waste"     "co"       "food"     "amp"     "reduce"    "co"  "https"   
2  "https"   "food"  "foodwaste"    "help"  "foodwaste"    "go"   "make"   

         7          8          9            10      11           12        13  \
0    "help"       "go"  "compost"  "foodwaste"   "get"        "eat"  "people"   
1    "make"     "make"       "go"       "year"    "go"  "recycling"    "year"   
2  "reduce"  "compost"      "eat"         "go"  "week"        "bag"    "save"   

         14      15      16      17           18      19  
0     "get"  "year"  "take"  "take"       "week"   "use"  
1  "people"   "get"  "save"   "use"       "time"  "time"  
2    "year"  "good"   "use"  "save"  "recycling"  "week"  


In [ ]:
dh[0]=dh[0].astype('category').cat.codes
dh[1]=dh[1].astype('category').cat.codes
dh[2]=dh[2].astype('category').cat.codes
dh[3]=dh[3].astype('category').cat.codes
dh[4]=dh[4].astype('category').cat.codes
dh[5]=dh[5].astype('category').cat.codes
dh[6]=dh[6].astype('category').cat.codes
dh[7]=dh[7].astype('category').cat.codes
dh[8]=dh[8].astype('category').cat.codes
dh[9]=dh[9].astype('category').cat.codes
dh[10]=dh[10].astype('category').cat.codes
dh[11]=dh[11].astype('category').cat.codes
dh[12]=dh[12].astype('category').cat.codes
dh[13]=dh[13].astype('category').cat.codes
dh[14]=dh[14].astype('category').cat.codes
dh[15]=dh[15].astype('category').cat.codes
dh[16]=dh[16].astype('category').cat.codes
dh[17]=dh[17].astype('category').cat.codes
dh[18]=dh[18].astype('category').cat.codes
dh[19]=dh[19].astype('category').cat.codes
dh.corr()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.5,0.5,0.5,1.0,1.0,0.5,0.5,1.0,0.5,-1.0,-0.5,0.5,-0.5,-0.5
1,-1.0,1.0,1.0,1.0,-1.0,1.0,-0.5,-0.5,-0.5,-1.0,-1.0,-0.5,-0.5,-1.0,-0.5,1.0,0.5,-0.5,0.5,0.5
2,-1.0,1.0,1.0,1.0,-1.0,1.0,-0.5,-0.5,-0.5,-1.0,-1.0,-0.5,-0.5,-1.0,-0.5,1.0,0.5,-0.5,0.5,0.5
3,-1.0,1.0,1.0,1.0,-1.0,1.0,-0.5,-0.5,-0.5,-1.0,-1.0,-0.5,-0.5,-1.0,-0.5,1.0,0.5,-0.5,0.5,0.5
4,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.5,0.5,0.5,1.0,1.0,0.5,0.5,1.0,0.5,-1.0,-0.5,0.5,-0.5,-0.5
5,-1.0,1.0,1.0,1.0,-1.0,1.0,-0.5,-0.5,-0.5,-1.0,-1.0,-0.5,-0.5,-1.0,-0.5,1.0,0.5,-0.5,0.5,0.5
6,0.5,-0.5,-0.5,-0.5,0.5,-0.5,1.0,1.0,-0.5,0.5,0.5,1.0,-0.5,0.5,1.0,-0.5,0.5,-0.5,-1.0,0.5
7,0.5,-0.5,-0.5,-0.5,0.5,-0.5,1.0,1.0,-0.5,0.5,0.5,1.0,-0.5,0.5,1.0,-0.5,0.5,-0.5,-1.0,0.5
8,0.5,-0.5,-0.5,-0.5,0.5,-0.5,-0.5,-0.5,1.0,0.5,0.5,-0.5,1.0,0.5,-0.5,-0.5,-1.0,1.0,0.5,-1.0
9,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.5,0.5,0.5,1.0,1.0,0.5,0.5,1.0,0.5,-1.0,-0.5,0.5,-0.5,-0.5


In [ ]:
from gensim.models import TfidfModel,LsiModel
tfidf = TfidfModel(gensim_corpus)
corpus_tfidf = tfidf[gensim_corpus]

def compute_coherence_UMass(corpus, dictionary, k):
   lsi_model = LsiModel(corpus=corpus_tfidf, num_topics=k)
   coherence = CoherenceModel(model=hdp_model,
                              corpus=corpus,
                              dictionary=dictionary,
                              coherence='u_mass')
   return coherence.get_coherence()
coherenceList_UMass = []
numTopicsList = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for k in numTopicsList:
   c_UMass = compute_coherence_UMass(corpus_tfidf, gensim_dictionary, 20)
   coherenceList_UMass.append(c_UMass)

NameError: ignored